# Coffee Shop Revenue Prediction - Complete ML Pipeline

**Toàn bộ quy trình**: EDA → Train Models → Optuna Tuning → Model Packaging

**Dataset**: coffee_shop_revenue1.csv (2,000 rows, 6 features)

**Mục tiêu**: Dự đoán doanh thu hàng ngày (Daily_Revenue)

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import xgboost as xgb
import lightgbm as lgb
import optuna
import pickle
import warnings
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries imported successfully")

## 2. Load Data

In [ ]:
# Load dataset
df = pd.read_csv('coffee_shop_revenue1.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")

df.head(10)

## 3. Exploratory Data Analysis (EDA)

### 3.1 Basic Information

In [ ]:
print("Data Info:")
print(df.info())
print("\n" + "="*80)

print("\nMissing Values:")
print(df.isnull().sum())
print("\n" + "="*80)

print("\nBasic Statistics:")
df.describe()

### 3.2 Target Variable Analysis

In [ ]:
print("Daily Revenue Statistics:")
print(f"Mean:   ${df['Daily_Revenue'].mean():.2f}")
print(f"Median: ${df['Daily_Revenue'].median():.2f}")
print(f"Std:    ${df['Daily_Revenue'].std():.2f}")
print(f"Min:    ${df['Daily_Revenue'].min():.2f}")
print(f"Max:    ${df['Daily_Revenue'].max():.2f}")
print(f"Range:  ${df['Daily_Revenue'].max() - df['Daily_Revenue'].min():.2f}")

### 3.3 Correlation Analysis

In [ ]:
# Correlation with target
correlations = df.corr()['Daily_Revenue'].sort_values(ascending=False)
print("Correlation with Daily_Revenue:")
print(correlations)

print("\n" + "="*80)
print("Top 3 Most Important Features:")
top_features = correlations[1:4]
for i, (feature, corr) in enumerate(top_features.items(), 1):
    print(f"{i}. {feature}: {corr:.4f}")

### 3.4 Visualizations

In [ ]:
# Create comprehensive visualization
fig = plt.figure(figsize=(20, 12))

# 1. Revenue Distribution
ax1 = plt.subplot(3, 3, 1)
df['Daily_Revenue'].hist(bins=50, edgecolor='black', alpha=0.7)
ax1.axvline(df['Daily_Revenue'].mean(), color='r', linestyle='--', 
            label=f'Mean: ${df["Daily_Revenue"].mean():.0f}')
ax1.axvline(df['Daily_Revenue'].median(), color='g', linestyle='--', 
            label=f'Median: ${df["Daily_Revenue"].median():.0f}')
ax1.set_title('Daily Revenue Distribution', fontsize=12, fontweight='bold')
ax1.set_xlabel('Daily Revenue ($)')
ax1.set_ylabel('Frequency')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Correlation Heatmap
ax2 = plt.subplot(3, 3, 2)
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, ax=ax2, cbar_kws={'shrink': 0.8})
ax2.set_title('Feature Correlation Matrix', fontsize=12, fontweight='bold')

# 3. Box Plot
ax3 = plt.subplot(3, 3, 3)
bp = ax3.boxplot(df['Daily_Revenue'], vert=True, patch_artist=True)
bp['boxes'][0].set_facecolor('skyblue')
ax3.set_ylabel('Daily Revenue ($)')
ax3.set_title('Revenue Box Plot', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3)

# 4-9. Scatter plots
features = [col for col in df.columns if col != 'Daily_Revenue']
for idx, feature in enumerate(features, start=4):
    ax = plt.subplot(3, 3, idx)
    ax.scatter(df[feature], df['Daily_Revenue'], alpha=0.5, s=10)
    
    z = np.polyfit(df[feature], df['Daily_Revenue'], 1)
    p = np.poly1d(z)
    ax.plot(df[feature], p(df[feature]), "r--", alpha=0.8, linewidth=2)
    
    corr = df[feature].corr(df['Daily_Revenue'])
    ax.set_xlabel(feature.replace('_', ' '))
    ax.set_ylabel('Daily Revenue ($)')
    ax.set_title(f'{feature.replace("_", " ")}\n(Corr: {corr:.3f})', 
                fontsize=10, fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/eda_comprehensive.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ EDA visualizations saved")

### 3.5 Outlier Detection

In [ ]:
Q1 = df['Daily_Revenue'].quantile(0.25)
Q3 = df['Daily_Revenue'].quantile(0.75)
IQR = Q3 - Q1
outliers = df[(df['Daily_Revenue'] < Q1 - 1.5*IQR) | (df['Daily_Revenue'] > Q3 + 1.5*IQR)]

print(f"Outliers detected: {len(outliers)} ({len(outliers)/len(df)*100:.2f}%)")
print(f"Outlier range: ${outliers['Daily_Revenue'].min():.2f} to ${outliers['Daily_Revenue'].max():.2f}")

## 4. Data Preparation

In [ ]:
# Separate features and target
X = df.drop('Daily_Revenue', axis=1)
y = df['Daily_Revenue']

feature_names = X.columns.tolist()

print(f"Features ({len(feature_names)}): {feature_names}")
print(f"Target: Daily_Revenue")
print(f"Shape: X={X.shape}, y={y.shape}")

## 5. Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

print(f"Train set: {len(X_train)} samples")
print(f"Test set:  {len(X_test)} samples")
print(f"\nTrain revenue: ${y_train.mean():.2f} ± ${y_train.std():.2f}")
print(f"Test revenue:  ${y_test.mean():.2f} ± ${y_test.std():.2f}")

## 6. Feature Scaling

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

with open('models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("✓ Features scaled and scaler saved")

## 7. Baseline Model Training (Default Hyperparameters)

In [ ]:
print("Training baseline models with default hyperparameters...\n")

# Linear Regression
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)
r2_lr = r2_score(y_test, y_pred_lr)
mape_lr = mean_absolute_percentage_error(y_test, y_pred_lr) * 100
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))

# Random Forest
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
r2_rf = r2_score(y_test, y_pred_rf)
mape_rf = mean_absolute_percentage_error(y_test, y_pred_rf) * 100
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))

# XGBoost
xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42, n_jobs=-1)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
r2_xgb = r2_score(y_test, y_pred_xgb)
mape_xgb = mean_absolute_percentage_error(y_test, y_pred_xgb) * 100
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))

# LightGBM
lgb_model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42, n_jobs=-1, verbose=-1)
lgb_model.fit(X_train, y_train)
y_pred_lgb = lgb_model.predict(X_test)
r2_lgb = r2_score(y_test, y_pred_lgb)
mape_lgb = mean_absolute_percentage_error(y_test, y_pred_lgb) * 100
rmse_lgb = np.sqrt(mean_squared_error(y_test, y_pred_lgb))

# Results
baseline_results = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest', 'XGBoost', 'LightGBM'],
    'R2': [r2_lr, r2_rf, r2_xgb, r2_lgb],
    'MAPE': [mape_lr, mape_rf, mape_xgb, mape_lgb],
    'RMSE': [rmse_lr, rmse_rf, rmse_xgb, rmse_lgb]
}).sort_values('R2', ascending=False)

print("\n" + "="*80)
print("BASELINE MODELS (Default Hyperparameters)")
print("="*80)
print(baseline_results.to_string(index=False))
print("="*80)

## 8. Hyperparameter Tuning with Optuna

Sử dụng Optuna để tối ưu hóa hyperparameters cho 3 models: LightGBM, XGBoost, Random Forest

In [ ]:
print("\n" + "="*80)
print("OPTUNA HYPERPARAMETER OPTIMIZATION")
print("="*80)
print("Đang tối ưu hóa hyperparameters cho 3 models...")
print("Mỗi model sẽ chạy 50 trials với 5-fold cross-validation")
print("="*80 + "\n")

### 8.1 LightGBM Optimization

In [ ]:
def objective_lightgbm(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'num_leaves': trial.suggest_int('num_leaves', 15, 127),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
        'random_state': 42,
        'n_jobs': -1,
        'verbose': -1
    }
    model = lgb.LGBMRegressor(**params)
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2', n_jobs=-1)
    return scores.mean()

print("[1] Optimizing LightGBM (50 trials)...")
study_lgb = optuna.create_study(direction='maximize', study_name='LightGBM')
study_lgb.optimize(objective_lightgbm, n_trials=50, show_progress_bar=True)

print(f"\n✓ Best LightGBM CV R²: {study_lgb.best_value:.6f}")
print(f"✓ Best params: {study_lgb.best_params}")

### 8.2 XGBoost Optimization

In [ ]:
def objective_xgboost(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 5.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
        'random_state': 42,
        'n_jobs': -1
    }
    model = xgb.XGBRegressor(**params)
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2', n_jobs=-1)
    return scores.mean()

print("\n[2] Optimizing XGBoost (50 trials)...")
study_xgb = optuna.create_study(direction='maximize', study_name='XGBoost')
study_xgb.optimize(objective_xgboost, n_trials=50, show_progress_bar=True)

print(f"\n✓ Best XGBoost CV R²: {study_xgb.best_value:.6f}")
print(f"✓ Best params: {study_xgb.best_params}")

### 8.3 Random Forest Optimization

In [ ]:
def objective_rf(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'random_state': 42,
        'n_jobs': -1
    }
    model = RandomForestRegressor(**params)
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2', n_jobs=-1)
    return scores.mean()

print("\n[3] Optimizing Random Forest (50 trials)...")
study_rf = optuna.create_study(direction='maximize', study_name='RandomForest')
study_rf.optimize(objective_rf, n_trials=50, show_progress_bar=True)

print(f"\n✓ Best Random Forest CV R²: {study_rf.best_value:.6f}")
print(f"✓ Best params: {study_rf.best_params}")

## 9. Train Tuned Models

In [ ]:
print("\n" + "="*80)
print("TRAINING MODELS WITH OPTIMIZED HYPERPARAMETERS")
print("="*80)

# Train LightGBM with best params
print("\n[1] Training LightGBM with best params...")
lgb_tuned = lgb.LGBMRegressor(**study_lgb.best_params, random_state=42, n_jobs=-1, verbose=-1)
lgb_tuned.fit(X_train, y_train)
y_pred_lgb_tuned = lgb_tuned.predict(X_test)
r2_lgb_tuned = r2_score(y_test, y_pred_lgb_tuned)
mape_lgb_tuned = mean_absolute_percentage_error(y_test, y_pred_lgb_tuned) * 100
rmse_lgb_tuned = np.sqrt(mean_squared_error(y_test, y_pred_lgb_tuned))
print(f"   R²:   {r2_lgb_tuned:.4f}")
print(f"   MAPE: {mape_lgb_tuned:.2f}%")
print(f"   RMSE: ${rmse_lgb_tuned:.2f}")

# Train XGBoost with best params
print("\n[2] Training XGBoost with best params...")
xgb_tuned = xgb.XGBRegressor(**study_xgb.best_params, random_state=42, n_jobs=-1)
xgb_tuned.fit(X_train, y_train)
y_pred_xgb_tuned = xgb_tuned.predict(X_test)
r2_xgb_tuned = r2_score(y_test, y_pred_xgb_tuned)
mape_xgb_tuned = mean_absolute_percentage_error(y_test, y_pred_xgb_tuned) * 100
rmse_xgb_tuned = np.sqrt(mean_squared_error(y_test, y_pred_xgb_tuned))
print(f"   R²:   {r2_xgb_tuned:.4f}")
print(f"   MAPE: {mape_xgb_tuned:.2f}%")
print(f"   RMSE: ${rmse_xgb_tuned:.2f}")

# Train Random Forest with best params
print("\n[3] Training Random Forest with best params...")
rf_tuned = RandomForestRegressor(**study_rf.best_params, random_state=42, n_jobs=-1)
rf_tuned.fit(X_train, y_train)
y_pred_rf_tuned = rf_tuned.predict(X_test)
r2_rf_tuned = r2_score(y_test, y_pred_rf_tuned)
mape_rf_tuned = mean_absolute_percentage_error(y_test, y_pred_rf_tuned) * 100
rmse_rf_tuned = np.sqrt(mean_squared_error(y_test, y_pred_rf_tuned))
print(f"   R²:   {r2_rf_tuned:.4f}")
print(f"   MAPE: {mape_rf_tuned:.2f}%")
print(f"   RMSE: ${rmse_rf_tuned:.2f}")

## 10. Compare Tuned vs Baseline Models

In [ ]:
# Tuned results
tuned_results = pd.DataFrame({
    'Model': ['XGBoost', 'LightGBM', 'Random Forest'],
    'R2': [r2_xgb_tuned, r2_lgb_tuned, r2_rf_tuned],
    'MAPE': [mape_xgb_tuned, mape_lgb_tuned, mape_rf_tuned],
    'RMSE': [rmse_xgb_tuned, rmse_lgb_tuned, rmse_rf_tuned]
}).sort_values('R2', ascending=False)

print("\n" + "="*80)
print("TUNED MODELS COMPARISON")
print("="*80)
print(tuned_results.to_string(index=False))
print("="*80)

# Save tuned results
tuned_results.to_csv('results/tuned_model_comparison.csv', index=False)
print("\n✓ Tuned results saved to results/tuned_model_comparison.csv")

# Improvement comparison
print("\n" + "="*80)
print("IMPROVEMENT FROM TUNING")
print("="*80)
print(f"XGBoost:       {r2_xgb:.4f} → {r2_xgb_tuned:.4f} ({(r2_xgb_tuned-r2_xgb)*100:+.2f}% improvement)")
print(f"LightGBM:      {r2_lgb:.4f} → {r2_lgb_tuned:.4f} ({(r2_lgb_tuned-r2_lgb)*100:+.2f}% improvement)")
print(f"Random Forest: {r2_rf:.4f} → {r2_rf_tuned:.4f} ({(r2_rf_tuned-r2_rf)*100:+.2f}% improvement)")
print("="*80)

## 11. Save Best Tuned Model

In [ ]:
# Select best tuned model
best_model_name = tuned_results.iloc[0]['Model']
models_dict = {
    'XGBoost': xgb_tuned,
    'LightGBM': lgb_tuned,
    'Random Forest': rf_tuned
}
best_model = models_dict[best_model_name]

# Save best tuned model
with open('models/best_model_tuned.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# Save model info
model_info = {
    'model_name': best_model_name,
    'metrics': tuned_results.iloc[0].to_dict(),
    'feature_names': feature_names
}

with open('models/model_info_tuned.pkl', 'wb') as f:
    pickle.dump(model_info, f)

# Save best params
best_params_dict = {
    'LightGBM': study_lgb.best_params,
    'XGBoost': study_xgb.best_params,
    'Random Forest': study_rf.best_params
}

with open('models/best_params.pkl', 'wb') as f:
    pickle.dump(best_params_dict, f)

print("\n" + "="*80)
print("MODEL PACKAGING COMPLETE")
print("="*80)
print(f"\n🏆 Best Model: {best_model_name}")
print(f"   R²:   {tuned_results.iloc[0]['R2']:.4f}")
print(f"   MAPE: {tuned_results.iloc[0]['MAPE']:.2f}%")
print(f"   RMSE: ${tuned_results.iloc[0]['RMSE']:.2f}")

print(f"\n📁 Files saved:")
print(f"   - models/best_model_tuned.pkl")
print(f"   - models/model_info_tuned.pkl")
print(f"   - models/best_params.pkl")
print(f"   - models/scaler.pkl")
print("="*80)

## 12. Feature Importance

In [ ]:
# Feature importance for best model
importances = best_model.feature_importances_
feature_imp = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values('Importance', ascending=False)

print("\nFeature Importance (Best Model):")
print("="*80)
for i, row in feature_imp.iterrows():
    bar_length = int(row['Importance'] * 50)
    bar = '█' * bar_length
    print(f"{row['Feature']:<40} {bar} {row['Importance']:.4f}")
print("="*80)

# Save
feature_imp.to_csv('results/feature_importance.csv', index=False)

# Visualize
plt.figure(figsize=(10, 6))
plt.barh(feature_imp['Feature'], feature_imp['Importance'], color='mediumpurple')
plt.xlabel('Importance', fontsize=12)
plt.title(f'Feature Importance - {best_model_name}', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('results/feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

## 13. Model Evaluation Visualizations

In [ ]:
# Get predictions from best model
y_pred_best = models_dict[best_model_name].predict(X_test)

fig = plt.figure(figsize=(18, 10))

# 1. Baseline vs Tuned - R²
ax1 = plt.subplot(2, 3, 1)
comparison_data = pd.DataFrame({
    'Model': ['XGBoost', 'LightGBM', 'Random Forest'],
    'Baseline': [r2_xgb, r2_lgb, r2_rf],
    'Tuned': [r2_xgb_tuned, r2_lgb_tuned, r2_rf_tuned]
})
x = np.arange(len(comparison_data))
width = 0.35
ax1.barh(x - width/2, comparison_data['Baseline'], width, label='Baseline', alpha=0.8)
ax1.barh(x + width/2, comparison_data['Tuned'], width, label='Tuned', alpha=0.8)
ax1.set_yticks(x)
ax1.set_yticklabels(comparison_data['Model'])
ax1.set_xlabel('R² Score')
ax1.set_title('Baseline vs Tuned - R² Score', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Tuned Models - MAPE
ax2 = plt.subplot(2, 3, 2)
bars = ax2.barh(tuned_results['Model'], tuned_results['MAPE'], color='lightcoral')
bars[0].set_color('gold')
ax2.set_xlabel('MAPE (%)')
ax2.set_title('Tuned Models - MAPE', fontweight='bold')
for i, v in enumerate(tuned_results['MAPE']):
    ax2.text(v, i, f' {v:.2f}%', va='center')
ax2.grid(True, alpha=0.3)

# 3. Tuned Models - RMSE
ax3 = plt.subplot(2, 3, 3)
bars = ax3.barh(tuned_results['Model'], tuned_results['RMSE'], color='lightgreen')
bars[0].set_color('gold')
ax3.set_xlabel('RMSE ($)')
ax3.set_title('Tuned Models - RMSE', fontweight='bold')
for i, v in enumerate(tuned_results['RMSE']):
    ax3.text(v, i, f' ${v:.2f}', va='center')
ax3.grid(True, alpha=0.3)

# 4. Predictions vs Actual
ax4 = plt.subplot(2, 3, 4)
ax4.scatter(y_test, y_pred_best, alpha=0.5, s=20)
ax4.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
ax4.set_xlabel('Actual Revenue ($)')
ax4.set_ylabel('Predicted Revenue ($)')
ax4.set_title(f'Predictions vs Actual - {best_model_name}\n(R²={tuned_results.iloc[0]["R2"]:.4f})',
              fontweight='bold')
ax4.grid(True, alpha=0.3)

# 5. Residual Plot
ax5 = plt.subplot(2, 3, 5)
residuals = y_test - y_pred_best
ax5.scatter(y_pred_best, residuals, alpha=0.5, s=20)
ax5.axhline(y=0, color='r', linestyle='--', lw=2)
ax5.set_xlabel('Predicted Revenue ($)')
ax5.set_ylabel('Residuals ($)')
ax5.set_title(f'Residual Plot - {best_model_name}', fontweight='bold')
ax5.grid(True, alpha=0.3)

# 6. Error Distribution
ax6 = plt.subplot(2, 3, 6)
ax6.hist(residuals, bins=30, edgecolor='black', alpha=0.7)
ax6.axvline(x=0, color='r', linestyle='--', lw=2)
ax6.set_xlabel('Residuals ($)')
ax6.set_ylabel('Frequency')
ax6.set_title('Error Distribution', fontweight='bold')
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/model_evaluation.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Model evaluation visualizations saved")

## 14. Making Predictions

In [ ]:
# Example prediction
example_data = pd.DataFrame({
    'Number_of_Customers_Per_Day': [300],
    'Average_Order_Value': [7.5],
    'Operating_Hours_Per_Day': [12],
    'Number_of_Employees': [8],
    'Marketing_Spend_Per_Day': [250.0],
    'Location_Foot_Traffic': [600]
})

print("Example Input:")
print(example_data.T)

prediction = best_model.predict(example_data)[0]
print(f"\n💰 Predicted Daily Revenue: ${prediction:,.2f}")

## 15. How to Use Saved Model

In [ ]:
print("""Để sử dụng model đã lưu:

```python
import pickle
import pandas as pd

# Load model
with open('models/best_model_tuned.pkl', 'rb') as f:
    model = pickle.load(f)

# Load model info
with open('models/model_info_tuned.pkl', 'rb') as f:
    info = pickle.load(f)

print(f"Model: {info['model_name']}")
print(f"R²: {info['metrics']['R2']:.4f}")

# Make prediction
data = pd.DataFrame({
    'Number_of_Customers_Per_Day': [300],
    'Average_Order_Value': [7.5],
    'Operating_Hours_Per_Day': [12],
    'Number_of_Employees': [8],
    'Marketing_Spend_Per_Day': [250.0],
    'Location_Foot_Traffic': [600]
})

prediction = model.predict(data)[0]
print(f"Predicted Revenue: ${prediction:.2f}")
```
""")

## 16. Final Summary

In [ ]:
print("\n" + "="*80)
print("COFFEE SHOP REVENUE PREDICTION - FINAL SUMMARY")
print("="*80)

print(f"\n📊 Dataset:")
print(f"   Total samples: {len(df):,}")
print(f"   Features: {len(feature_names)}")
print(f"   Target: Daily_Revenue")
print(f"   Train/Test: {len(X_train)}/{len(X_test)} (80/20 split)")

print(f"\n🏆 Best Model: {best_model_name} (Tuned)")
print(f"   R² Score:  {tuned_results.iloc[0]['R2']:.4f}")
print(f"   MAPE:      {tuned_results.iloc[0]['MAPE']:.2f}%")
print(f"   RMSE:      ${tuned_results.iloc[0]['RMSE']:.2f}")

print(f"\n⭐ Top 3 Most Important Features:")
for i, row in feature_imp.head(3).iterrows():
    print(f"   {i+1}. {row['Feature']}: {row['Importance']*100:.2f}%")

print(f"\n📈 Improvement from Optuna Tuning:")
print(f"   XGBoost:       {r2_xgb:.4f} → {r2_xgb_tuned:.4f} ({(r2_xgb_tuned-r2_xgb)*100:+.2f}%)")
print(f"   LightGBM:      {r2_lgb:.4f} → {r2_lgb_tuned:.4f} ({(r2_lgb_tuned-r2_lgb)*100:+.2f}%)")
print(f"   Random Forest: {r2_rf:.4f} → {r2_rf_tuned:.4f} ({(r2_rf_tuned-r2_rf)*100:+.2f}%)")

print(f"\n📁 Generated Files:")
print(f"   Models:")
print(f"   - models/best_model_tuned.pkl")
print(f"   - models/model_info_tuned.pkl")
print(f"   - models/best_params.pkl")
print(f"   - models/scaler.pkl")
print(f"   \n   Results:")
print(f"   - results/tuned_model_comparison.csv")
print(f"   - results/feature_importance.csv")
print(f"   - results/eda_comprehensive.png")
print(f"   - results/model_evaluation.png")
print(f"   - results/feature_importance.png")

print("\n" + "="*80)
print("✅ TOÀN BỘ PIPELINE HOÀN THÀNH - MODEL SẴN SÀNG SỬ DỤNG")
print("="*80)